In [1]:
import numpy as np
import random
from datetime import datetime
import pandas as pd

import sys
import matplotlib.pyplot as plt
from deap import base
from deap import creator
from deap import tools

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.metrics import confusion_matrix 
from sklearn import preprocessing
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.externals.six import StringIO
import os
import scikitplot as skplt
from matplotlib import pyplot
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

/home/nakayama/.local/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
df = pd.read_csv('Datasets/credit.csv')

In [ ]:
X_test = df_test.iloc[:,1:196]
y_test = df_test.loc[:, df_test.columns == 'Y']

In [ ]:
#!pip install deap

#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

vector_size = 196

#X_test, y_test = load_svmlight_file('CC132/test.txt')
#X_test = X_test.toarray()

#X, y = load_svmlight_file('CC132/train.txt')
#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5)

#X_train =  X_train.toarray()
#X_val = X_val.toarray()

graph = []
graph2 = []


def evalOneMax(individual):
    
    rr = 0.0
    soma = sum(individual)
    
    if (soma >0):
        mask = []
        for i in range(len(individual)):
            if(individual[i] == 0):
                mask.append(i)
    
        #print mask
        print ("Features removed: ", len(individual) - soma )
   
        subX_train = np.delete(X_train,mask, axis=1)
        subX_val = np.delete(X_val,mask, axis=1)
     
        clf  = Perceptron()
        #clf  = LinearDiscriminantAnalysis()
        #clf = RandomForestClassifier(n_estimators=100)
        clf.fit(subX_train, y_train.ravel())
        
        # predicao do classificador    
        y_pred = clf.predict(subX_val) 

        # mostra o resultado do classificador na base de teste
        rr =  clf.score(subX_val, y_val)
        print ("Rec Rate ",  rr )

        graph.append(1.-rr)
        graph2.append(soma)
        


    return rr,


def FinalTest(individual):
    
    
    print ("Testing best individual on the Test Set..." )
    clf  = Perceptron()
    #clf  = LinearDiscriminantAnalysis()
    #clf = RandomForestClassifier(n_estimators=100)

    clf.fit(X_train, y_train.ravel())
    
    y_pred = clf.predict(X_test) 
    rr =  clf.score(X_test, y_test)
    print ("Performance Using all features ", rr)
    
    mask = []
    for i in range(len(individual)):
        if(individual[i] == 0):
            mask.append(i)
       
    subX_train = np.delete(X_train,mask, axis=1)
    subX_test = np.delete(X_test,mask, axis=1)
     
    clf  = Perceptron()
    #clf  = LinearDiscriminantAnalysis()
    #clf = RandomForestClassifier(n_estimators=100)

    clf.fit(subX_train, y_train.ravel())
    
    # predicao do classificador    
    y_pred = clf.predict(subX_test)
    
    # mostra o resultado do classificador na base de teste
    rr =  clf.score(subX_test, y_test)
    print ("Performance Using best individual ", rr)
    return (1.-rr)        
    
    roc_score = roc_auc_score(y_test, subX_test)
    print("ROC AUC=%.3f", roc_score)
    roc_plot = roc_curve(y_test, subX_test)
    pyplot.plot(roc_plot, linestyle='dotted')
    pyplot.xlabel('False Positive Rate')
    pyplot.ylabel('True Positive Rate')
    pyplot.show()


    
 

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
# Attribute generator
toolbox.register("attr_bool", random.randint, 0, 1)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, vector_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


# Operator registering
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
        

        
#############################################################
#############################################################        
def main():
    
    random.seed(datetime.now())
    
    
    pop = toolbox.population(n=20)
    CXPB, MUTPB, NGEN = 0.8, 0.1, 1
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))
    
    # Begin the evolution
    for g in range(NGEN):
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        #invalid_ind = [ind for ind in offspring]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
            #print ind, fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
        
        #graph.append(max(fits))
        #graph2.append(len(individual))
        
        
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, Rec Rate: %s Nb of Features: %s" % (best_ind, best_ind.fitness.values, sum(best_ind)   ))

    err = FinalTest(best_ind)
    plt.plot(graph2, graph, 'ro')
    plt.plot(sum(best_ind), err, 'bo')

    plt.axis([30,vector_size-30,0,0.25])
    
    plt.xlabel("Qtde de Caracteristicas")
    plt.ylabel("Erro de Classificacao")
 
    #line =plt.plot(graph)
    plt.show()
    
    

if __name__ == "__main__":

    main()
   




In [ ]:
print(y_pred)